In [ ]:
%load_ext autoreload
%autoreload 2

from pandas_datareader import data as wb
import os
import pandas as pd

stocks = pd.read_csv('stocks.csv').fillna(0)
stocks = stocks.set_index('Unnamed: 0')

for s in stocks:
    d = wb.DataReader(s, data_source='yahoo', start='1997-1-1')
    file_path = os.path.join('data', s+'.csv')
    d.to_csv(file_path)

In [ ]:
# Manual input
usd_to_sgd_rate = 1.3250            # USD -> SGD
remaining_usd_cash = 1362.46+25.62  # Remaining USD cash in IB and DBS

In [ ]:
def load_data(tick):
    file_path = os.path.join('data', tick+'.csv')
    data = pd.read_csv(file_path)
    dt_idx = pd.DatetimeIndex(data['Date'])
    data.index = dt_idx

    return data

stock_buy_transactions = pd.read_csv('stock_buy_transactions.csv')
stock_dividends = pd.read_csv('stock_dividends.csv')

df = pd.DataFrame()
df['tick'] = stock_buy_transactions['tick'].drop_duplicates()

df = pd.merge(df, stock_dividends.groupby(['tick']).sum(), on='tick', how='outer')
df = df.rename(columns={'amount':'dividends'})
df['dividends'] = df['dividends'].fillna(0)

stock_buy_transactions['total_cost'] = stock_buy_transactions['buy_price']*stock_buy_transactions['quantity']\
    +stock_buy_transactions['commission']
df = pd.merge(df, stock_buy_transactions.groupby(['tick']).sum(), on='tick', how='outer')
df['avg_cost'] = df['total_cost']/df['quantity']
df = df.drop(columns=['Unnamed: 0', 'buy_price'])

df['currency_sgd'] = 'SGD'
df['current_price'] = 0.0
for index, row in df.iterrows():
    t = row['tick']
    d = load_data(t)
    current_price = d['Adj Close'].iloc[-1]
    df.at[index, 'current_price'] = current_price
    df.at[index, 'currency_sgd'] = 1
    if stocks[t]['currency']=='USD':
        df.at[index, 'currency_sgd'] = usd_to_sgd_rate

df['pl'] = df['current_price']*df['quantity']+df['dividends']-df['total_cost']
df['pl_percent'] = (df['current_price']*df['quantity']+df['dividends'])/df['total_cost']-1.0
df['current_value_sgd'] = df['current_price']*df['quantity']*df['currency_sgd']
df['allocation'] = df['current_value_sgd']/df['current_value_sgd'].sum()

df_vis = pd.DataFrame()
df_vis['tick'] = df['tick']
df_vis['dividends'] = df['dividends']
df_vis['quantity'] = df['quantity']
df_vis['commission'] = df['commission']
df_vis['total_cost'] = df['total_cost'].map('{:,.2f}'.format)
df_vis['avg_cost'] = df['avg_cost'].map('{:,.2f}'.format)
df_vis['current_price'] = df['current_price'].map('{:,.2f}'.format)
df_vis['pl'] = df['pl'].map('{:,.2f}'.format)
df_vis['pl_percent'] = df['pl_percent'].map('{:,.2%}'.format)
df_vis['current_value_sgd'] = df['current_value_sgd']
df_vis['allocation'] = df['allocation'].map('{:,.2%}'.format)

df_vis

In [ ]:
stock_others = pd.read_csv('stock_others.csv') # TODO: not calculated yet

print('SG stocks PL:')
print(df[df['currency_sgd']==1]['pl'].sum())
print('\nUS stocks PL (in SGD):')
print(df[df['currency_sgd']!=1]['pl'].sum()*usd_to_sgd_rate)
print('\nTotal PL (in SGD)')
print(df[df['currency_sgd']==1]['pl'].sum()+df[df['currency_sgd']!=1]['pl'].sum()*usd_to_sgd_rate)

In [ ]:
from datetime import datetime, timedelta

currency_exchange_transactions = pd.read_csv('currency_exchange_transactions.csv')
currency_exchange_transactions = currency_exchange_transactions.set_index('Unnamed: 0')
total_original_sgd_to_usd = currency_exchange_transactions['amount'].sum()
print('Total Converted SGD to USD\t' + str(total_original_sgd_to_usd))

total_sgd_cost = df[df['currency_sgd']==1]['total_cost'].sum()
print('Total SGD Stocks Cost\t\t' + str(total_sgd_cost) + '\n')

usd_total_remain_sgd = df[df['currency_sgd']!=1]['current_value_sgd'].sum()+remaining_usd_cash*usd_to_sgd_rate
sgd_pl = df[df['currency_sgd']==1]['pl'].sum()
now = datetime.now()
stock_buy_transactions['expense_ratio']=0.0
stock_buy_transactions['days'] = stock_buy_transactions['time'].apply(\
                                                            lambda x: (now-datetime.strptime(x, '%Y-%m-%d')).days)
for index, row in stock_buy_transactions.iterrows():
    stock_buy_transactions.at[index, 'expense_ratio'] = stocks[row['tick']]['expense_ratio']
stock_buy_transactions['expense']=stock_buy_transactions['days']/365*stock_buy_transactions['expense_ratio']\
                                    *stock_buy_transactions['total_cost']/100*usd_to_sgd_rate
total_expense = stock_buy_transactions['expense'].sum()
actual_pl = usd_total_remain_sgd-total_original_sgd_to_usd+sgd_pl-total_expense

print('Actual Total PL (in SGD):\t' + str(actual_pl))
print('Actual Total PL%\t\t' + '{:.2%}'.format(actual_pl/(total_original_sgd_to_usd+total_sgd_cost)))

stock_buy_transactions.loc[(stock_buy_transactions.currency == 'SGD'),'currency']=1.0
stock_buy_transactions.loc[(stock_buy_transactions.currency == 'USD'),'currency']=usd_to_sgd_rate
stock_buy_transactions['annualized_cost_sgd'] = stock_buy_transactions['currency']*stock_buy_transactions['days']\
                                                /365*stock_buy_transactions['total_cost']
annualized_cost_sgd = stock_buy_transactions['annualized_cost_sgd'].sum()
print(annualized_cost_sgd/(total_original_sgd_to_usd+total_sgd_cost))
print(actual_pl/annualized_cost_sgd)

In [ ]:
transfer_to_ib_transactions = pd.read_csv('transfer_to_ib_transactions.csv')
transfer_to_ib_transactions

total_transfered_sgd = transfer_to_ib_transactions[transfer_to_ib_transactions.currency=='SGD']['amount'].sum()

currency_exchange_transactions['coverted_amount'] = currency_exchange_transactions['amount']/\
                            currency_exchange_transactions['rate']
converted_sgd = currency_exchange_transactions[currency_exchange_transactions.place=='ib']\
                            [currency_exchange_transactions.currency=='SGDUSD']['amount'].sum()
print(converted_sgd)
converted_usd = currency_exchange_transactions[currency_exchange_transactions.place=='ib']\
                            [currency_exchange_transactions.currency=='SGDUSD']['coverted_amount'].sum()
print(converted_usd)
stock_sgd_purchase = stock_buy_transactions[stock_buy_transactions.currency==1]\
    [stock_buy_transactions['commission']<5]['total_cost'].sum()
print(stock_sgd_purchase)
currency_exchange_transactions
stock_buy_transactions

print(total_transfered_sgd-converted_sgd-stock_sgd_purchase) # this number should be very close

# in USD
print(converted_usd+stock_sgd_purchase/usd_to_sgd_rate)